In [1]:
from IPython.core.display import HTML
from pymongo import MongoClient
from bs4 import BeautifulSoup
from requests import get

# Web Scraping Demo

To show the workflow for scraping a page we will try scraping information off [indeed](http://www.indeed.com/).

As an example, let's try searching for 'Data Scientist' in Denver, CO.  From going directly to the website and searching for this, we see that the URL is 'http://www.indeed.com/jobs?q=Data+Scientist&l=Denver%2C+CO'.  

Let's use `requests.get()` to retrieve the raw HTML.

In [2]:
url = 'http://www.indeed.com/jobs?q=Data+Scientist&l=Denver%2C+CO'
response = get(url)

print response.status_code

200


---

We can see that the search was successful because we recieved a `200` status code.  For a complete breakdown of the various status codes you may encounter, refer to [this](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).

---

Let's try printing out the raw HTML...

In [3]:
print response.content

<!DOCTYPE html>
<html>
<head>
<meta http-equiv="content-type" content="text/html;charset=UTF-8">
<!-- pll --><script type="text/javascript" src="/s/7c5ca65/en_US.js"></script>
<link href="/s/2a09b15/jobsearch_all.css" rel="stylesheet" type="text/css">
<link rel="alternate" type="application/rss+xml" title="Data Scientist Jobs, Employment in Denver, CO" href="http://rss.indeed.com/rss?q=Data+Scientist&l=Denver%2C+CO">
<link rel="alternate" media="handheld" href="/m/jobs?q=Data+Scientist&l=Denver%2C+CO">
<script type="text/javascript">
    
    window['closureReadyCallbacks'] = [];

    function call_when_jsall_loaded(cb) {
        if (window['closureReady']) {
            cb();
        } else {
            window['closureReadyCallbacks'].push(cb);
        }
    }
</script>
<script type="text/javascript" src="/s/00e999e/jobsearch-all-compiled.js"></script>
            <script type="text/javascript">
var pingUrlsForGA = [];

var searchUID = '1ag6omh1bah6qao4';
var tk = '1ag6omh1bah6qao4';

---

This is really gross and, unless you're a robot, you will have a hard time parsing out the useful information.  iPython has a nice `core.display.HTML` method for displaying what this would actually look like in a browser.  Let's take a look at that now...

---

In [4]:
HTML(response.content)

## BeautifulSoup to the rescue!

BeautifulSoup is the package that will allow you to quickly, and efficiently parse this HTML into only the data that we are concerned with.

In [5]:
soup = BeautifulSoup(response.content, 'html.parser')

print soup.prettify()

<!DOCTYPE html>
<html>
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type">
   <!-- pll -->
   <script src="/s/7c5ca65/en_US.js" type="text/javascript">
   </script>
   <link href="/s/2a09b15/jobsearch_all.css" rel="stylesheet" type="text/css">
    <link href="http://rss.indeed.com/rss?q=Data+Scientist&amp;l=Denver%2C+CO" rel="alternate" title="Data Scientist Jobs, Employment in Denver, CO" type="application/rss+xml">
     <link href="/m/jobs?q=Data+Scientist&amp;l=Denver%2C+CO" media="handheld" rel="alternate">
      <script type="text/javascript">
       window['closureReadyCallbacks'] = [];

    function call_when_jsall_loaded(cb) {
        if (window['closureReady']) {
            cb();
        } else {
            window['closureReadyCallbacks'].push(cb);
        }
    }
      </script>
      <script src="/s/00e999e/jobsearch-all-compiled.js" type="text/javascript">
      </script>
      <script type="text/javascript">
       var pingUrlsForGA = [];

var se

---

Slightly nicer, but this is still way more than we need...  From here we resort to Chrome development tools!

If we are concerned with finding the job titles we could right click on the title and click 'Inspect.' This will bring up the developer tools and will immediately take you to the relevent portion of the HTML.  From here you will find the element that encompasses everything that we want to grab (notice how when you hover over a particular element it highlights what is being defined on the page).  In this case we should look to an element that looks like this...

```html
<div class="  row  result" id="p_7f192716571c8758" data-jk="7f192716571c8758" itemscope="" itemtype="http://schema.org/JobPosting" data-tn-component="organicJob"> ...
```

i.e. we are interested in a `div` element whose `class` contains `row`.  Let's go ahead and select all elements which satisfy this...

**NOTE**: For a more comprehensive breakdown of using Chrome development tools refer to the `web_scraping.md` file

---

In [6]:
postings = soup.findAll('div', attrs={'class': 'row'})

# Similarly we can find which contained element has the title that we want
# In this case it's a 'a' element with an 'data-tn-element' = 'jobTitle'
titles = [post.find('a', attrs={'data-tn-element': 'jobTitle'}).text for post in postings]
print titles

[u'Statewide Data Director', u'Medicare and Medicaid Data Analyst', u'DATA SCIENTIST', u'Data Scientist', u'Senior Data Scientist', u'Spec Statistical Rsrch', u'Data Scientist', u'Data Scientist 3', u'Data Scientist', u'Data Science Analyst', u'Data Scientist', u'Statistician/Data Scientist', u'Data Scientist Role', u'Research Scientist [Exp. with Machine Learning & Computer Vi...', u'Research Analyst I']


---

Yay!  We actually have something intelligible now!  Let's try extracting everything in one go using a function.

---

In [7]:
def extract_post(post):
    '''
    INPUT:
        post: bs4.element.Tag
    OUTPUT:
        insert: dict
    '''
    try:
        title = post.find('a', attrs={'data-tn-element': 'jobTitle'}).text
    except:
        print 'ERROR EXTRACTING TITLE'
        return None
    try:
        loc = post.find('span', attrs={'class': 'location'}).text
    except:
        print 'ERROR EXTRACTING LOCATION'
        return None
    try:
        desc = post.find('span', attrs={'class': 'summary'}).text
    except:
        print 'ERROR EXTRACTING DESCRIPTION'
        return None
    insert = {'title': title,
              'location': loc,
              'description': desc}
    return insert

---

Notice all the `try ... except` blocks?  This will ensure that our whole program doesn't fail if one of these extractions doesn't pass.  
You should, however, **be very careful when using these** and make sure that legitimate errors aren't getting skipped over when employing this technique.  
**Never** jump straight to using a `try ... except` block because you may be erroneously escaping legitimate errors.  
Once you are satisfied that you've squashed any pesky bugs, these `try ... except` blocks will ensure that one website being constructed slightly different won't bring your script screeching to a halt.

What about dynamically passing different search queries and locations?  Let's throw this bad boy into a function like so...

---

In [8]:
def query_indeed(job_title, city, state):
    '''
    INPUT:
        job_title: str
        city: str
        state: str
    OUTPUT:
        requests.models.Response
    '''
    job_title = job_title.replace(' ', '+')
    url = 'http://www.indeed.com/jobs?q={0}&l={1}%2C+{2}'.format(job_title, city, state)
    response = get(url)
    if response.status_code != 200:
        print 'WARNING', response.status_code
    else:
        return response

In [9]:
# Create a instance of the MongoClient
client = MongoClient()

# Initialize the database
db = client['test_database']

# Initialize table
tab = db['job_desc']

In [10]:
# Query Indeed.com searching for 'Data Scientist' in Denver, CO
html = query_indeed('Data Scientist', 'Denver', 'CO')

# Create BeautifulSoup object with the resulting HTML
soup = BeautifulSoup(html.content, 'html.parser')

# Grab only the tags associated with job postings
postings = soup.findAll('div', attrs={'class': 'row'})

# Extract info for each job posting in postings
inserts = [extract_post(post) for post in postings]

# We can now insert each of these records one at a time like so...
for insert in inserts:
    tab.insert_one(insert)

# Or we can insert all of them at once...
# tab.insert_many(inserts)

---
# NPR API

To illustrate working with APIs we will be working through a toy example using NPR's public API.  In order to use their API you need to first [create an account](http://www.npr.org/account/login?returnUrl=http%3A%2F%2Fwww.npr.org%2F).  Once you've created your account go in to manage your information and scroll down to the very bottom where your API key will be listed under the developer information section.  See below for an example of what you are looking for...

![](./images/npr_api.png)

This also provides you a link to their [documentation](http://www.npr.org/api/index) as well as an easy way to regenerate a new key in case you do something foolish with it like post it inside of a lecture.  We can then set this save this key as an environment variable (or json file, please refer to the `web_scraping.md` guide for more information on how and why to do this) to use inside a script.  To reiterate, **DO NOT HARDCODE THIS KEY IN YOUR CODE**.  While the NPR API is very nice in that you don't have to jump through any hoops to be issued one of these keys, not all APIs are this friendly.

Let's say we wanted to get information in JSON format for any stories concerning Data Science from March 21st, 2016 to March 25, 2016.  We could do that with the following query...

`http://api.npr.org/query?id=1003&requiredAssets=text&startDate=2016-03-21&endDate=2016-03-25&searchTerm=Data%20Science&dateType=story&output=JSON&apiKey=[Your_API_KEY]`

NPR also provides an easy to use [tool for generating specific queries](http://www.npr.org/api/queryGenerator.php).  Once you generate a few different queries, it's easy to get a feel for what each of these fields is doing.  Let's break it down...

* `http://api.npr.org/query?`
    * This is the base of every query to the NPR API
* `id=1003`
    * This id tells the API which topics you are interested in.  In this case `id=1003` selects just the U.S. topic.  `id=1003,1004` would select query both the U.S. and World topics
* `requiredAssets=text`
    * This says we only want results that have text associated with the story
* `startDate=2016-03-21` and `endDate=2016-03-25`
    * These dictate the starting and ending dates of the search in YYYY-MM-DD format.  If we only wanted to search on a specific date we could replace these with just `date=YYYY-MM-DD`
* `searchTerm=Data%20Science`
    * This is the string to search for.  Notice that spaces are replaced with `%20`
* `dateType=story`
    * This dictates that we only want stories that were published on a particular date rather than having a correction issued on that date
* `output=JSON`
    * This is the data format that we would like as the response
* `apiKey=[Your_API_KEY]`
    * This is where you pass your API Key in to provide authentication

Now that we know how this query is working, let's take a look at what we get back!

In [1]:
import os
from requests import get

# My key is saved as an environment variable titled API_ACCESS_KEY
API_KEY = os.environ['NPR_ACCESS_KEY']

In [2]:
# Define our query to the API
url = 'http://api.npr.org/query?id=1003&requiredAssets=text&startDate=2016-03-21&endDate=2016-03-25'\
      '&searchTerm=Data%20Science&dateType=story&output=JSON&apiKey={0}'.format(API_KEY)

# Use requests.get to query the API
response = get(url)

# Save the JSON result to a variable
result = response.json()

---

Using the power of the `params` argument to the `requests.get` function, we can format the previous call in a much more readable and flexible way.  The following code produces the exact same output...

---

In [4]:
# Define the base URL (up to the ? of the previous call)
url = 'http://api.npr.org/query'

# Define what information should be passed along
payload = {
    'id': '1003',
    'requiredAssets': 'text',
    'startDate': '2016-03-21',
    'endDate': '2016-03-25',
    'searchTerm': 'Data Science',
    'dateType': 'story',
    'output': 'JSON',
    'apiKey': API_KEY,
    'this-key-is-ignored': None
}

# Use requests.get to query the API with the payload
response = get(url, params=payload)

print response.url, '\n'

result2 = response.json()
print result == result2

http://api.npr.org/query?startDate=2016-03-21&apiKey=MDIxNTg5NDEzMDE0NjA0OTg3Nzk5NzJjMg000&endDate=2016-03-25&searchTerm=Data+Science&output=JSON&requiredAssets=text&dateType=story&id=1003 

True


In [2]:
result

{u'list': {u'link': [{u'$text': u'http://api.npr.org/query?id=1003&searchTerm=Data%20Science&apiKey=MDIxNTg5NDEzMDE0NjA0OTg3Nzk5NzJjMg000',
    u'type': u'api'}],
  u'miniTeaser': {},
  u'story': [{u'audioRunByDate': {},
    u'byline': [{u'id': u'471580887',
      u'link': [{u'$text': u'http://www.npr.org/people/409798174/danielle-kurtzleben?ft=nprml&f=1003',
        u'type': u'html'},
       {u'$text': u'http://api.npr.org/query?id=409798174&meta=inherit&apiKey=MDIxNTg5NDEzMDE0NjA0OTg3Nzk5NzJjMg000',
        u'type': u'api'}],
      u'name': {u'$text': u'Danielle Kurtzleben ',
       u'personId': u'409798174'}}],
    u'id': u'471578401',
    u'image': [{u'caption': {u'$text': u'Supporters listen as Republican presidential candidate Donald Trump speaks at a campaign event at the I-X Center earlier this month in Cleveland, Ohio.'},
      u'copyright': {},
      u'crop': [{u'height': u'2273',
        u'src': u'https://media.npr.org/assets/img/2016/03/24/gettyimages-515144074-89cbc3f88338

As we can see, the result is massive.  To make sense of what this passed back we have to start drilling down into the many layers to parse out what we want.  As you drill down into each successive layer it's important to constantly be checking what datatype we are working with!  If it's a `list`, then take a look at just the first element.  If it's a `dict`, check to see what keys we available and try and get a sense for what each contains.

Drilling down to find what we want can be a frustrating process, but the nice thing about APIs are that they won't change (as much).

Just remember, it's `dicts` all the way down!

![](./images/turtles.jpeg)

In [3]:
type(result)

dict

In [4]:
result.keys()

[u'version', u'list']

In [5]:
result['version']

u'0.94'

---

Looks like the `list` key contains all the information we want.  Let's check what the type of `result['list']` is before printing it out...

---

In [6]:
type(result['list'])

dict

In [7]:
result['list'].keys()

[u'teaser', u'miniTeaser', u'link', u'story', u'title']

In [8]:
result['list']['teaser']

{u'$text': u'NPR coverage of national news, U.S. politics, elections, business, arts, culture, health and science, and technology. Subscribe to the NPR Nation RSS feed.'}

---

Stepping through each key lists various information about your query including the original query contained in `link` and the topics in `title`.  Looks like the `story` key contains all the juicy information!

---

In [9]:
type(result['list']['story'])

list

In [10]:
# Let's check the type of just the first element of that list
type(result['list']['story'][0])

dict

In [25]:
# Now let's see what kind of keys this dict has
result['list']['story'][0].keys()

[u'miniTeaser',
 u'subtitle',
 u'text',
 u'storyDate',
 u'partnerId',
 u'keywords',
 u'textWithHtml',
 u'id',
 u'byline',
 u'pubDate',
 u'title',
 u'image',
 u'audioRunByDate',
 u'thumbnail',
 u'parent',
 u'priorityKeywords',
 u'lastModifiedDate',
 u'teaser',
 u'link',
 u'slug',
 u'shortTitle',
 u'organization']

---

Now we are getting somewhere!  Looks like we have drilled down into the output for a single article.  Let's get a sense for what is contained in each of these keys...

---

In [11]:
result['list']['story'][0]['title']

{u'$text': u"Stopping Trump Won't Be So Easy. Here's Why, In One Chart"}

In [12]:
result['list']['story'][0]['pubDate']

{u'$text': u'Thu, 24 Mar 2016 12:46:00 -0400'}

In [13]:
result['list']['story'][0]['teaser']

{u'$text': u"Donald Trump has a base that skews heavily toward early deciders. His opponents end up fighting over who's left."}

In [14]:
result['list']['story'][0]['text']

{u'paragraph': [{u'$text': u"Here's one of the biggest strengths Donald Trump has going for him: His voters decide, and they stay decided.",
   u'num': u'1'},
  {u'$text': u"That matters as the GOP race moves to Wisconsin and beyond. Ted Cruz and John Kasich are looking to thwart Trump's chance, but they may be too late to change many voters' minds in those places.",
   u'num': u'2'},
  {u'$text': u'According to exit polling data, most GOP primary voters decide on a candidate well before they vote, and the earliest-deciding voters have thus far almost always leaned toward Trump.',
   u'num': u'3'},
  {u'$text': u"Clearly, Trump is in a race of his own on this count. To a remarkable degree, his support skews toward people who decided earlier. The below chart (using exit polling data compiled by John Coleman, political science professor at the University of Minnesota) shows the difference between a candidate's share of people who decided in the last few days and his share of those who de

In [15]:
result['list']['story'][0]['byline']

[{u'id': u'471580887',
  u'link': [{u'$text': u'http://www.npr.org/people/409798174/danielle-kurtzleben?ft=nprml&f=1003',
    u'type': u'html'},
   {u'$text': u'http://api.npr.org/query?id=409798174&meta=inherit&apiKey=MDIxNTg5NDEzMDE0NjA0OTg3Nzk5NzJjMg000',
    u'type': u'api'}],
  u'name': {u'$text': u'Danielle Kurtzleben ', u'personId': u'409798174'}}]

In [16]:
result['list']['story'][0]['link']

[{u'$text': u'http://www.npr.org/2016/03/24/471578401/stopping-trump-won-t-be-so-easy-here-s-why-in-one-chart?ft=nprml&f=1003',
  u'type': u'html'},
 {u'$text': u'http://api.npr.org/query?id=471578401&apiKey=MDIxNTg5NDEzMDE0NjA0OTg3Nzk5NzJjMg000',
  u'type': u'api'},
 {u'$text': u'http://n.pr/1pyC6MY', u'type': u'short'}]

Try exploring what information the rest of the keys contain before moving on...

---

It looks like all the information is still nested in each of these responses though.  Let's try parsing out the information to get the article text:

In [17]:
result['list']['story'][0]['text'].keys()

[u'paragraph']

In [18]:
result['list']['story'][0]['text']['paragraph']

[{u'$text': u"Here's one of the biggest strengths Donald Trump has going for him: His voters decide, and they stay decided.",
  u'num': u'1'},
 {u'$text': u"That matters as the GOP race moves to Wisconsin and beyond. Ted Cruz and John Kasich are looking to thwart Trump's chance, but they may be too late to change many voters' minds in those places.",
  u'num': u'2'},
 {u'$text': u'According to exit polling data, most GOP primary voters decide on a candidate well before they vote, and the earliest-deciding voters have thus far almost always leaned toward Trump.',
  u'num': u'3'},
 {u'$text': u"Clearly, Trump is in a race of his own on this count. To a remarkable degree, his support skews toward people who decided earlier. The below chart (using exit polling data compiled by John Coleman, political science professor at the University of Minnesota) shows the difference between a candidate's share of people who decided in the last few days and his share of those who decided before that.",


In [19]:
result['list']['story'][0]['text']['paragraph'][0]

{u'$text': u"Here's one of the biggest strengths Donald Trump has going for him: His voters decide, and they stay decided.",
 u'num': u'1'}

In [20]:
result['list']['story'][0]['text']['paragraph'][0]['$text']

u"Here's one of the biggest strengths Donald Trump has going for him: His voters decide, and they stay decided."

---
This is a perfect use case for list comprehension!

---

In [21]:
print '  '.join([line['$text'] for line in result['list']['story'][0]['text']['paragraph']])

Here's one of the biggest strengths Donald Trump has going for him: His voters decide, and they stay decided.  That matters as the GOP race moves to Wisconsin and beyond. Ted Cruz and John Kasich are looking to thwart Trump's chance, but they may be too late to change many voters' minds in those places.  According to exit polling data, most GOP primary voters decide on a candidate well before they vote, and the earliest-deciding voters have thus far almost always leaned toward Trump.  Clearly, Trump is in a race of his own on this count. To a remarkable degree, his support skews toward people who decided earlier. The below chart (using exit polling data compiled by John Coleman, political science professor at the University of Minnesota) shows the difference between a candidate's share of people who decided in the last few days and his share of those who decided before that.  In all 20 of the states for which there are exit polling data except one (Cruz's home state of Texas), Trump ha

---

Let's create a function that will, given a article's JSON information, parse any information we are interested in and create a dictionary ready to be inserted into a Mongo Database.  For the purposes of this example we will extract the title, article text, url, date published, author, and source.

---

In [22]:
def extract_info(article):
    '''
    INPUT: dict object with output from the api
    OUTPUT: dict object to insert into mongodb
    '''
    headline = article['title']['$text']
    date_published = str(article['pubDate']['$text'])
    try:
        author = [str(author['name']['$text']).strip() for author in article['byline']]
    except:
        author = None
    url = str(article['link'][0]['$text'])
    article_text = ' '.join([line.get('$text', '\n') for line in article['text']['paragraph']])
    source = [source['orgAbbr'] for source in article['organization']]
    insert = {'url': url,
              'headline': headline,
              'source': source,
              'date_published': date_published,
              'author': author,
              'article_text': article_text}
    return insert

In [23]:
articles = [extract_info(article) for article in result['list']['story']]
articles[0]

{'article_text': u'Here\'s one of the biggest strengths Donald Trump has going for him: His voters decide, and they stay decided. That matters as the GOP race moves to Wisconsin and beyond. Ted Cruz and John Kasich are looking to thwart Trump\'s chance, but they may be too late to change many voters\' minds in those places. According to exit polling data, most GOP primary voters decide on a candidate well before they vote, and the earliest-deciding voters have thus far almost always leaned toward Trump. Clearly, Trump is in a race of his own on this count. To a remarkable degree, his support skews toward people who decided earlier. The below chart (using exit polling data compiled by John Coleman, political science professor at the University of Minnesota) shows the difference between a candidate\'s share of people who decided in the last few days and his share of those who decided before that. In all 20 of the states for which there are exit polling data except one (Cruz\'s home state

In [24]:
articles[1]

{'article_text': u'Funny how feelings about sleep change over the years. Many children fight bedtime and are still getting up once or more during the night well into childhood. Meantime, adults often feel they can never get enough sleep and, if they\'re anything like me, have vivid fantasies about napping. Now a study suggests that parents\' own sleep quality may bias how they perceive their child\'s sleep issues. The study, conducted in Finland, looked at 100 children ages 2 to 6. Their parents filled out questionnaires about the sleep quality of their kids and also of themselves. And for a week, the children wore an actigraph, which is a wearable band that measures movement and can estimate periods of sleep. Think of it as a souped-up Fitbit. The authors found that parents with their own sleep problems more frequently reported that their children had sleeping difficulties, even when the actigraph data suggested differently. "People who sleep poorly overestimate their children\'s slee

In [25]:
articles[2]

{'article_text': u'Experiencing the world as a different gender than the one assigned to you at birth can take a toll. Nearly all research into transgender individuals\' mental health shows poorer outcomes. A study looking specifically at transgender women, predominantly women of color, only further confirms that reality. What\'s less clear, however, is whether trans individuals experience more mental distress due to external factors, such as discrimination and lack of support, or internal factors, such as gender dysphoria, the tension resulting from having a gender identity that differs from the one assigned at birth. Transgender people are often treated extremely poorly by their parents, by their schools, by society at large, and that can lead to problems in school and at work, as well as poverty and increased risk of substance use, according to Ilana Sherer, assistant medical director of the Child and Adolescent Gender Center at the University of California Benioff Children\'s Hospi

---

Try writing your own function to be able to pass in different values to the API query such as different search terms and dates!

---

In [ ]:
def single_query(searchterm, start_date, end_date):
    ''' Performs a single NPR API query and returns the article data
    INPUT:
        searchterm - str to search for
        start_date - date str in 'YYYY-MM-DD' format
        end_date - date str in 'YYYY-MM-DD' format
    OUTPUT:
        articles - list of dicts containing article information
    '''
    pass